Import Necessary Modules

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob


Load data 

In [37]:
df_news = pd.read_csv("../data/raw_analyst_ratings.csv") # news related data 
df_price = pd.read_csv("../data/price_all_tickers.csv") # price related data that includes all tickers

In [38]:
df_price.rename(columns={'Date': 'date'}, inplace=True) # Renaming the Date column to date for merging purpose later
df_price.head() # view data


,date,Close,High,Low,Open,Volume,Ticker
0,2009-01-02,2.721686,2.730385,2.554037,2.575630,746015200,AAPL
1,2009-01-05,2.836553,2.884539,2.780469,2.794266,1181608400,AAPL
2,2009-01-06,2.789767,2.914229,2.770872,2.877641,1289310400,AAPL
3,2009-01-07,2.729484,2.774170,2.706990,2.753477,753048800,AAPL
4,2009-01-08,2.780169,2.793666,2.700393,2.712090,673500800,AAPL


Normalize the dates of both dataframes

In [40]:
df_news['date']= pd.to_datetime(df_news['date'], format='mixed', utc=True).dt.tz_convert(None)
df_price['date']= pd.to_datetime(df_price['date'], format='mixed', utc=True).dt.tz_convert(None)

In [42]:
df_news['date'].head()

0   2020-06-05 14:30:54
1   2020-06-03 14:45:20
2   2020-05-26 08:30:07
3   2020-05-22 16:45:06
4   2020-05-22 15:38:59
Name: date, dtype: datetime64[ns]

Sentiment Analysis using TextBlob

In [41]:
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # polarity between -1 (negative) and 1 (positive) with 0 being neutral

df_news['sentiment'] = df_news['headline'].astype(str).apply(get_sentiment)
df_news.head()

,Unnamed: 0,headline,url,publisher,date,stock,sentiment
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 14:30:54,A,0.0
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 14:45:20,A,0.0
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 08:30:07,A,0.0
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 16:45:06,A,0.0
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 15:38:59,A,0.0


In [15]:
print(df_news["sentiment"].value_counts())

sentiment
 0.000000    934928
 0.500000     86313
 0.250000     31492
 0.136364     16654
 0.100000     15469
              ...  
-0.170635         1
 0.041818         1
-0.047500         1
-0.155804         1
 0.026452         1
Name: count, Length: 5133, dtype: int64


From the above value counts it can be seen that a huge majority of the sentiment lies within the neutral (0) segment

In [45]:
# Given that there are multiple publishings made within a day at varying times grouping the sentiment by date using the mean was done
df_daily_Sent = df_news.groupby('date')['sentiment'].mean().reset_index()
df_daily_Sent.rename(columns={'sentiment': 'avg_daily_sentiment'}, inplace=True)
df_daily_Sent.head()

,date,avg_daily_sentiment
0,2009-02-14,0.000000
1,2009-04-27,0.000000
2,2009-04-29,0.000000
3,2009-05-22,0.000000
4,2009-05-27,0.234091


Compute Daily Stock returns

In [ ]:
df_price = df_price.sort_values('date')
df_price['daily_return'] = df_price['Close'].pct_change() # daily percent change of the Close price
df_daily_return = df_price[['date', 'daily_return']] # a separate dataframe is created including date and calculated daily return
df_daily_return.head()

,date,daily_return
0,2009-01-02,NaN
3774,2009-01-02,-0.001354
14245,2009-01-02,4.472030
18019,2009-01-02,-0.986576
7548,2009-01-02,38.812354


In [47]:
df_merge = pd.merge(df_daily_Sent, df_daily_return, on='date', how='inner') # dataframe merging using date column when record is found in both dataframes
df_merge.head()

,date,avg_daily_sentiment,daily_return
0,2009-04-27,0.0,-0.757126
1,2009-04-27,0.0,1.552243
2,2009-04-27,0.0,-0.564698
3,2009-04-27,0.0,-0.939220
4,2009-04-27,0.0,58.486938


Correlation Analysis

In [48]:
correlation = df_merge['avg_daily_sentiment'].corr(df_merge['daily_return'])
print("Correlation between sentiment and stock returns:", correlation)

Correlation between sentiment and stock returns: 0.012973224426202069


In [36]:
print(df_merge.head())
print("\n Pearson Correlation:", correlation)

        date  avg_daily_sentiment  daily_return
0 2009-04-27                  0.0     -0.757126
1 2009-04-27                  0.0      1.552243
2 2009-04-27                  0.0     -0.564698
3 2009-04-27                  0.0     -0.939220
4 2009-04-27                  0.0     58.486938

 Pearson Correlation: 0.012973224426202069
